In [66]:
#Import dependencies
import os
import pandas as pd
import time
from sqlalchemy import create_engine

In [26]:
#Name files
ExtractStart = time.time()
titlesFile = os.path.join("Resources","netflix_titles.csv")
# moviesStartTime = time.time()
moviesFile = os.path.join("Resources","movies_all_streaming.csv")
# tvshowsTime= time.time()
tvshowsFile = os.path.join("Resources","tv_shows_all_streaming.csv")

In [27]:
titles_df = pd.read_csv(titlesFile)
# titles_df.head()

movies_df = pd.read_csv(moviesFile)
# movies_df.head()

tvshows_df = pd.read_csv(tvshowsFile)
# tvshows_df.head()
ExtractEnd = time.time()
ExtractTime = ExtractEnd-ExtractStart
TransformStart = ExtractEnd

In [52]:
titlesClean = titles_df[
                        ["type","title","country","date_added","release_year","rating"]]
titlesClean = titlesClean.rename(columns={
    'release_year':'year'
})
titlesClean['date_Added'] = pd.to_datetime(titlesClean['date_Added'])
# titlesClean.head()
# titlesClean['Country'].count()

In [73]:
moviesClean = movies_df[
    ["Title","Year","Age","IMDb","Rotten Tomatoes","Netflix","Country"]
]

mC_all= len(moviesClean)

moviesClean = moviesClean.loc[moviesClean['Netflix']==1]
mC_Netflix = len(moviesClean)
moviesClean = moviesClean.drop(columns='Netflix')
moviesClean = moviesClean.rename(columns={
    'Title':'title',
    'Year':'year',
    'Age':'age',
    'IMDb':'imdb',
    'Rotten Tomatoes':"rotten_tomatoes",
    'Country':'country'})
# moviesClean.head()
# moviesClean['Country'].count()

In [61]:
movieMerge = pd.merge(titlesClean, moviesClean, on=['title','year'], how='inner')
mM_all = len(movieMerge)
movieMerge = movieMerge.dropna()
mM_complete = len(movieMerge)
print(mM_all)
print(mM_complete)
movieMerge = movieMerge.rename(columns={'country_x':"country"})
movieMerge = movieMerge.drop(columns='country_y')
movieMerge.reset_index(inplace=True,drop=True)
movieMerge

2528
763


,Type,Title,Country,Date_Added,Year,Rating,Age,IMDb,Rotten_Tomatoes
0,Movie,9,United States,2017-11-16,2009,PG-13,13+,7.1,57%
1,Movie,21,United States,2020-01-01,2008,PG-13,13+,6.8,36%
2,Movie,1922,United States,2017-10-20,2017,TV-MA,18+,6.3,90%
3,Movie,22-Jul,"Norway, Iceland, United States",2018-10-10,2018,R,18+,6.8,80%
4,Movie,13 Cameras,United States,2016-08-13,2015,NR,18+,5.1,77%
...,...,...,...,...,...,...,...,...,...
758,Movie,Young Adult,United States,2019-11-20,2011,R,18+,6.2,80%
759,Movie,Yucatán,Spain,2019-02-15,2018,TV-MA,7+,5.4,60%
760,Movie,Zack and Miri Make a Porno,United States,2018-10-01,2008,R,18+,6.5,65%
761,Movie,Zindagi Na Milegi Dobara,India,2019-12-15,2011,TV-14,7+,8.1,91%


In [62]:
tvshowsClean = tvshows_df[["Title","Year","Age","IMDb","Rotten Tomatoes","Netflix"]]

tvC_all= len(tvshowsClean)

tvshowsClean = tvshowsClean.loc[tvshowsClean['Netflix']==1]
tvC_Netflix = len(tvshowsClean)
tvshowsClean = tvshowsClean.drop(columns='Netflix')
tvshowsClean = tvshowsClean.rename(columns={
    'Title':'title',
    'Year':'year',
    'Age':'age',
    'IMDb':'imdb',
    'Rotten Tomatoes':"rotten_tomatoes"
    })
# tvshowsClean.head()

In [68]:
tvshowsMerge = pd.merge(titlesClean, tvshowsClean, on=['title','year'], how='inner')
tvM_all = len(tvshowsMerge)
tvshowsMerge = tvshowsMerge.dropna()
tvM_complete = len(tvshowsMerge)
print(tvM_all)
print(tvM_complete)
tvshowsMerge.reset_index(inplace=True,drop=True)
tvshowsMerge.dtypes
TransformEnd = time.time()
TransformTime = TransformEnd-TransformStart
LoadStart = TransformEnd

709
139


Type                       object
Title                      object
Country                    object
Date_Added         datetime64[ns]
Year                        int64
Rating                     object
Age                        object
IMDb                      float64
Rotten_Tomatoes            object
dtype: object

In [67]:
connection_string = "postgres:postgres@localhost:5432/AreYouStillWatching_db"
engine = create_engine(f'postgresql://{connection_string}')

In [71]:
engine.table_names()

['final']

In [72]:
movieMerge.to_sql(name='final', con=engine, if_exists='append', index=False)
tvshowsMerge.to_sql(name='final', con=engine, if_exists='append', index=False)

ProgrammingError: (psycopg2.errors.UndefinedColumn) column "Type" of relation "final" does not exist
LINE 1: INSERT INTO final ("Type", "Title", "Country", "Date_Added",...
                           ^

[SQL: INSERT INTO final ("Type", "Title", "Country", "Date_Added", "Year", "Rating", "Age", "IMDb", "Rotten_Tomatoes") VALUES (%(Type)s, %(Title)s, %(Country)s, %(Date_Added)s, %(Year)s, %(Rating)s, %(Age)s, %(IMDb)s, %(Rotten_Tomatoes)s)]
[parameters: ({'Type': 'Movie', 'Title': '9', 'Country': 'United States', 'Date_Added': datetime.datetime(2017, 11, 16, 0, 0), 'Year': 2009, 'Rating': 'PG-13', 'Age': '13+', 'IMDb': 7.1, 'Rotten_Tomatoes': '57%'}, {'Type': 'Movie', 'Title': '21', 'Country': 'United States', 'Date_Added': datetime.datetime(2020, 1, 1, 0, 0), 'Year': 2008, 'Rating': 'PG-13', 'Age': '13+', 'IMDb': 6.8, 'Rotten_Tomatoes': '36%'}, {'Type': 'Movie', 'Title': '1922', 'Country': 'United States', 'Date_Added': datetime.datetime(2017, 10, 20, 0, 0), 'Year': 2017, 'Rating': 'TV-

In [ ]:
LoadEnd = time.time()
LoadTime = LoadEnd-LoadStart